# Machine Translation

In [1]:
!pip install -q datasets
!pip insatll wandb
import wandb
wandb.login()
import os
from tokenizers import Tokenizer, pre_tokenizers, trainers, models
from datasets import load_dataset

ds = load_dataset("thainq107/iwslt2015-en-vi")

# word-based
tokenizer_en = Tokenizer(models.WordLevel(unk_token="<unk>"))
tokenizer_vi = Tokenizer(models.WordLevel(unk_token="<unk>"))

tokenizer_en.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer_vi.pre_tokenizer = pre_tokenizers.Whitespace()

trainer = trainers.WordLevelTrainer(
    vocab_size=15000,
    min_frequency=2,
    special_tokens=["<unk>", "<pad>", "<bos>", "<eos>"],
    show_progress=True
)

# train tokenizer
tokenizer_en.train_from_iterator(ds["train"]["en"], trainer=trainer)
tokenizer_vi.train_from_iterator(ds["train"]["vi"], trainer=trainer)

# tokenizer

tokenizer_en.save("tokenizer_en.json")
tokenizer_vi.save("tokenizer_vi.json")
len(ds['train']),len(ds['validation'])

ERROR: unknown command "insatll" - maybe you meant "install"


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phuochuunguyen2009 (phuochuunguyen2009-localcompany) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

validation-00000-of-00001.parquet:   0%|          | 0.00/181k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/133317 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1268 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

(133317, 1268)

ENCODER

In [2]:
from transformers import PreTrainedTokenizerFast

MAX_LEN = 75

# load tokenizer
tokenizer_en = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_en.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)
tokenizer_vi = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_vi.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)

def preprocess_function(examples):
  src_texts = examples['en']
  tgt_texts = ['<bos>' + sent + "<eos>" for sent in examples['vi']]

  src_encodings = tokenizer_en (
      src_texts,padding="max_length",truncation=True,max_length=MAX_LEN
  )
  tgt_encodings = tokenizer_vi (
      tgt_texts,padding="max_length",truncation=True,max_length=MAX_LEN
  )

  return {
      "input_ids":src_encodings["input_ids"],
      "labels":tgt_encodings["input_ids"]
  }
preprocessed_ds = ds.map(preprocess_function,batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Modeling

a. RNNS

In [3]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

class Seq2SeqRNNConfig(PretrainedConfig):
  def __init__(
      self,
      vocab_size_src=10000,
      vocab_size_tgt=10000,
      embedding_dim=128,
      hidden_size=128,
      dropout=0.1,
      **kwargs
  ):
    super().__init__(**kwargs)
    self.vocab_size_src = vocab_size_src
    self.vocab_size_tgt = vocab_size_tgt
    self.embedding_dim = embedding_dim
    self.hidden_size = hidden_size
    self.dropout = dropout
class EncoderRNN(nn.Module):
  def __init__(
      self,
      input_size,
      embedding_dim,
      hidden_size,
      dropout_p=0.1
  ):
    super().__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size,embedding_dim)
    self.gru = nn.GRU(embedding_dim,hidden_size,batch_first=True)
    self.dropout = nn.Dropout(dropout_p)
  def forward(self,x):
    embedded =  self.dropout(self.embedding(x))
    output,hidden = self.gru(embedded)
    return output, hidden
class DecoderRNN(nn.Module):
  def __init__(
      self,
      hidden_size,
      embedding_dim,
      output_size
  ):
    super().__init__()
    self.embedding = nn.Embedding(output_size,embedding_dim)
    self.gru = nn.GRU(embedding_dim,hidden_size,batch_first=True)
    self.out = nn.Linear(hidden_size,output_size)
  def forward(self, x,hidden):
    output = self.embedding(x)
    output,hidden = self.gru(output,hidden)
    output = self.out(output)
    return output,hidden
class Seq2SeqRNNModel(PreTrainedModel):
  config_class = Seq2SeqRNNConfig

  def __init__(self,config,tokenizer_en):
    super().__init__(config)
    self.encoder = EncoderRNN(
        config.vocab_size_src,config.embedding_dim,
        config.hidden_size,config.dropout
    )
    self.decoder = DecoderRNN(
        config.hidden_size,
        config.embedding_dim,
        config.vocab_size_tgt
    )
    self.BOS_IDX = tokenizer_en.bos_token_id
    self.loss_fn = nn.CrossEntropyLoss(ignore_index=0)
  def forward(self,input_ids,labels):
    batch_size,seq_len = labels.shape
    decoder_input = torch.full((batch_size,1),self.BOS_IDX,dtype=torch.long).to(input_ids.device)
    encoder_output,decoder_hidden = self.encoder(input_ids)
    decoder_outputs = []

    for i in range(seq_len):
      decoder_output,decoder_hidden = self.decoder(decoder_input,
                                                   decoder_hidden)
      decoder_outputs.append(decoder_output)
      decoder_input = labels[:,i].unsqueeze(1)
    logits = torch.cat(decoder_outputs,dim=1)
    loss = self.loss_fn(logits.view(-1,logits.shape[-1]),labels.view(-1))
    return {"loss":loss,"logits":logits}
config = Seq2SeqRNNConfig(
    vocab_size_src = len(tokenizer_en),
    vocab_size_tgt = len(tokenizer_vi),

)
model = Seq2SeqRNNModel(config,tokenizer_en)

transformers

In [14]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

def generate_square_subsequent_mask(sz,device):
  mask = (torch.triu(torch.ones((sz,sz),device=device))==1).transpose(0,1)
  mask = mask.float().masked_fill(mask==0,float("-inf")).masked_fill(mask==1,float(0.0))
  return mask
def create_mask(src,tgt):
  src_seq_len = src.shape[1]
  tgt_seq_len = tgt.shape[1]
  device = src.device

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len,device).to(torch.bool)
  src_mask = torch.zeros((src_seq_len,src_seq_len),device=device).type(torch.bool)
  src_padding_mask = (src==0)
  tgt_padding_mask = (tgt==0)
  return src_mask,tgt_mask,src_padding_mask,tgt_padding_mask
class Seq2SeqTransformerConfig(PretrainedConfig):
  def __init__(
    self,
    vocab_size_src = 10000,
    vocab_size_tgt=10000,
    max_seq_length=50,
    d_model = 256,
    num_heads=8,num_layers=2,dropout=0.1,
    **kwargs
  ):
    super().__init__(**kwargs)
    self.vocab_size_src = vocab_size_src
    self.vocab_size_tgt = vocab_size_tgt
    self.max_seq_length = max_seq_length
    self.d_model = d_model
    self.num_heads = num_heads
    self.num_layers = num_layers
    self.dropout = dropout
class Seq2SeqTransformerModel(PreTrainedModel):
  config_class = Seq2SeqTransformerConfig
  def __init__(self,config):
    super().__init__(config)

    self.embedding_src = nn.Embedding(config.vocab_size_src ,config.d_model)
    self.embedding_tgt = nn.Embedding(config.vocab_size_tgt,config.d_model)

    self.position_embedding_src = nn.Embedding(config.max_seq_length,config.d_model)
    self.position_embedding_tgt = nn.Embedding(config.max_seq_length,config.d_model)

    self.transformer = nn.Transformer(
        d_model=config.d_model,
        nhead=config.num_heads,
        num_encoder_layers = config.num_layers,
        num_decoder_layers=config.num_layers,
        dropout=config.dropout,
        batch_first=True
    )

    self.generator =nn.Linear(config.d_model,config.vocab_size_tgt)
    self.loss_fn =nn.CrossEntropyLoss(ignore_index=0) # ignore PAD token
  def forward(self,input_ids,labels):
    batch_size,seq_len_src = input_ids.shape
    _,seq_len_tgt = labels.shape

    src_positions = torch.arange(seq_len_src,device=input_ids.device).unsqueeze(0)
    tgt_positions = torch.arange(seq_len_tgt,device=labels.device).unsqueeze(0)

    src_embedded = self.embedding_src(input_ids) + self.position_embedding_src(src_positions)
    tgt_embedded = self.embedding_tgt(labels) + self.position_embedding_tgt(tgt_positions)

    src_mask,tgt_mask,src_key_padding_mask,tgt_key_padding_mask = create_mask(input_ids,labels)

    outs = self.transformer(
        src_embedded,tgt_embedded,src_mask,tgt_mask,src_key_padding_mask=src_key_padding_mask,tgt_key_padding_mask=tgt_key_padding_mask
    )

    logits = self.generator(outs)
    loss = self.loss_fn(logits.permute(0,2,1),labels)

    return {"loss":loss,"logits":logits}
  def encode(self,src,src_mask):
    seq_len_src = src.shape[1]
    src_positions = torch.arange(seq_len_src,device=src.device).unsqueeze(0)
    src_embedded = self.embedding_src(src)+self.position_embedding_src(src_positions)
    return self.transformer.encoder(src_embedded,src_mask)
  def decode(self,tgt,encoder_output,tgt_mask):
    seq_len_tgt = tgt.shape[1]
    tgt_positions = torch.arange(seq_len_tgt,device=tgt.device).unsqueeze(0)
    tgt_embedded = self.embedding_tgt(tgt)+self.position_embedding_tgt(tgt_positions)
    return self.transformer.decoder(tgt_embedded,encoder_output,tgt_mask)

config = Seq2SeqTransformerConfig(
    vocab_size_src = len(tokenizer_en),vocab_size_tgt=len(tokenizer_vi),max_seq_length=75
)
transformer_model = Seq2SeqTransformerModel(config)

test model

In [ ]:
input_ids = torch.tensor([preprocessed_ds['train'][0]['input_ids']])
labels = torch.tensor([preprocessed_ds['train'][0]['labels']])
pred = transformer_model(input_ids,labels)
pred

In [7]:
import os
from transformers import Trainer, TrainingArguments

In [ ]:

# train

training_args = TrainingArguments(
    output_dir = "./en-vi-machine-translation",
    logging_dir='logs',
    eval_strategy = 'epoch',
    save_strategy="epoch",
    logging_strategy = 'epoch',
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    num_train_epochs=25,
    learning_rate=2e-5,
    save_total_limit=1,
    report_to = "wandb"
)
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset=preprocessed_ds['train'],
    eval_dataset=preprocessed_ds['validation']
)
trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir = "./en-vi-machine-translation",
    logging_dir='logs',
    eval_strategy = 'epoch',
    save_strategy="epoch",
    logging_strategy = 'epoch',
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    num_train_epochs=25,
    learning_rate=2e-5,
    save_total_limit=1,
    report_to = "wandb"
)
trainer = Trainer(
    model=transformer_model,
    args = training_args,
    train_dataset=preprocessed_ds['train'],
    eval_dataset=preprocessed_ds['validation']
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.654100,2.444265
2,1.706700,1.200933
3,0.994800,0.786561
4,0.698000,0.572674
5,0.527500,0.441784
6,0.418100,0.355223
7,0.343500,0.294865
8,0.290600,0.251400
9,0.251600,0.219073
10,0.221900,0.194332


In [ ]:
!pip install sacrebleu

In [ ]:

def greedy_decode(model,src,src_mask,max_len,start_symbol, device="cpu"):
  src = src.to(device)
  src_mask = src_mask.to(device)

  memory = model.encode(src,src_mask)
  ys = torch.ones(1,1).fill_(start_symbol).type(torch.long).to(device)

  for i in range(max_len-1):
    memory = memory.to(device)
    tgt_mask = (generate_square_subsequent_mask(ys.size(1),device).type(torch.bool)).to(device)
    out = model.decode(ys,memory,tgt_mask)
    prob = model.generator(out[:,-1,:])
    _,next_word = torch.max(prob,dim=1)
    next_word = next_word[-1].item()

    ys = torch.cat([ys,torch.ones(1,1).type_as(src.data).fill_(next_word)],dim=1)
    if next_word ==3:
      break
  return ys
def translate(model,src_sentence,device):
  model.eval()
  input_ids = tokenizer_en([src_sentence],return_tensors="pt")['input_ids'].to(device)
  num_tokens = input_ids.shape[1]
  src_mask = (torch.zeros(num_tokens,num_tokens)).type(torch.bool).to(device)
  tgt_tokens = greedy_decode(
      model,input_ids,src_mask,max_len=num_tokens+5,start_symbol=2,device=device
  )
  return tokenizer_vi.decode(tgt_tokens.detach().cpu()[0])
translate(transformer_model,"i go to school",transformer_model.device)

from tqdm import tqdm
import sacrebleu

pred_sentences,tgt_sentences = [],[]
for sample in tqdm(ds['test']):
  src_sentence = sample['en']
  tgt_sentence=  sample['vi']
  pred_sentence = translate(transformer_model,src_sentence,transformer_model.device)
  pred_sentences.append(pred_sentence)
  tgt_sentences.append(tgt_sentence)
bleu_score =  sacrebleu.corpus_bleu(pred_sentences,tgt_sentences,force=True)
bleu_score